<a href="https://colab.research.google.com/github/1ucky40nc3/ml4me/blob/main/vision/frame-interpolation/film/run_film_frame-interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Run [FILM](https://github.com/google-research/frame-interpolation) Frame-Interpolation

## Set up the Colab Notebook

In [ ]:
# @title Mount your Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# @title Clone the [FILM Repository](https://github.com/google-research/frame-interpolation)
%cd /content
!git clone https://github.com/google-research/frame-interpolation

In [ ]:
# @title Install Python Requirements
!pip install parameterized
!pip install mediapy
!pip install apache-beam
!pip install google-cloud-bigquery-storage
!pip install natsort
!pip install gdown
!pip install av
!pip install einops

In [ ]:
# @title Download the Pretrained Models

# @markdown 1. Link this [Google Drive Folder](https://drive.google.com/drive/folders/1q8110-qp225asX3DQvZnfLfJPkCHmDpy?usp=sharing) into a your Google Drive

# @markdown 2. Copy the Pre-trained Model Files to this Colab Instance (automatic)

# @markdown ---

import os


# @markdown Provide the path in your GDrive to the pre-trained model files:
gdrive_pretrained_model_dir = "/content/gdrive/MyDrive/projects/ML4ME/vision/frame_interpolation/film/pretrained_models" # @param {type: "string"}
# @markdown Set the local path for a copy of the files:
local_pretrained_models_base_dir = "/content/"
local_pretrained_models_dir = os.path.join(local_pretrained_models_base_dir, "pretrained_models")

# Try to skip this step if the local dir exists
if not os.path.exists(local_pretrained_models_dir):
    !mkdir -p $local_pretrained_models_base_dir
    !cp -r $gdrive_pretrained_model_dir $local_pretrained_models_dir

In [ ]:
# @title Implement Utils

from typing import List

import os
import glob
from base64 import b64encode

import torch
import torchvision

import einops

from IPython.display import HTML
from IPython.display import display

from tqdm.auto import tqdm


VIDEO_MIME_TYPE_MAP = {
    ".avi": "video/x-msvideo",
    ".mp4": "video/mp4",
    ".mpeg": "video/mpeg"
}


def show_images(
    paths: List[str],
    data_url_prefix: str = "data:image/png;base64,",
    image_width: int = 300,
    sample_style: str = "display:flex;flex-direction:column",
    filename_style: str = "text-align:center;"
) -> None:
    data = [open(i, "rb").read() for i in paths]
    data_urls = [data_url_prefix + b64encode(i).decode() for i in data]

    configs = (sample_style, image_width, filename_style)
    sample = '<div stype="{}"><img src="%s" width={}/><p style="{}">{}</p></div>'
    display(HTML(
        f"""
        <div class="row" style="display:flex;flex-wrap:wrap;">
            {''.join([sample.format(*configs, path) for path in paths])}
        </div>
        """ % tuple(data_urls)
    ))


def frames_to_video(
    frames_pattern: str,
    video_path: str,
    fps: float=24.,
) -> None:
    video_dir = os.path.join(*os.path.split(video_path)[:-1])
    os.makedirs(video_dir, exist_ok=True)

    paths = sorted(glob.glob(frames_pattern))
    pbar = tqdm(paths, desc="Reading frames", total=len(paths))
    video_array = torch.stack([
        torchvision.io.read_image(p)
        for p in pbar
    ])
    video_array = einops.rearrange(
        video_array,
        "t c h w -> t h w c"
    )
    torchvision.io.write_video(
        video_path,
        video_array,
        fps=fps
    )


def show_video(
    filename: str,
    width: int=800,
    autoplay: bool=False
) -> None:
    """Display a video."""
    with open(filename, "rb") as file:
        data = file.read()

    mime = filename.split(".")[-1]
    data = b64encode(data).decode()
    data_url = f"data:video/{mime};base64,{data}"

    autoplay = "autoplay" if autoplay else ""
    html = f"""
    <video width={width} controls {autoplay}>
            <source src="{data_url}" type="video/mp4">
    </video>
    """
    display(HTML(html))


def pad(num: int, length: int, value: int=0) -> str:
    num_pad = length - len(str(num))
    padding = "".join([str(value)] * num_pad)
    return f"{padding}{num}"


def video_to_frames(
    video_path: str,
    frames_dir: str
) -> None:
    os.makedirs(frames_dir, exist_ok=True)
    video_frames, _, _ = torchvision.io.read_video(
        video_path,
        output_format="TCHW"
    )
    num_frames = video_frames.shape[0]
    num_chars = len(str(num_frames))
    pbar = tqdm(
        enumerate(video_frames),
        desc="Writing frames",
        total=num_frames
    )
    for i, frame in pbar:
        filename = f"frame_{pad(i, num_chars)}.png"
        path = os.path.join(frames_dir, filename)
        torchvision.io.write_png(frame, path)

## Interpolate!

In [ ]:
# @title Interpolate (once) between two frames
%cd /content/frame-interpolation

# @markdown Provide a starting frame for the interpolation
frame1 = "/content/frame-interpolation/photos/one.png" # @param {type: "string"}
# @markdown Give a target frame for the interpolation
frame2 = "/content/frame-interpolation/photos/two.png" # @param {type: "string"}
# @markdown Set a path for the interpolated frame
output_frame = "/content/frame-interpolation/photos/output_middle.png" # @param {type: "string"}


!python -m eval.interpolator_test \
   --frame1 $frame1 \
   --frame2 $frame2 \
   --model_path $local_pretrained_models_dir/film_net/Style/saved_model \
   --output_frame $output_frame

show_images([frame1, output_frame, frame2])

In [ ]:
# @title Interpolate (n-times) between frames
%cd /content/frame-interpolation

import os


# @markdown Provide the path to an input directory (as a glob pattern that finds directories with min. 2 frames):
input_dir = "/content/frame-interpolation/photos" # @param {type: "string"}
# @markdown Specify the number of interpolated frames:
n_times_to_interpolate = 6 # @param {type: "number"}

!python -m eval.interpolator_cli  \
   --pattern $input_dir \
   --model_path $local_pretrained_models_dir/film_net/Style/saved_model \
   --times_to_interpolate $n_times_to_interpolate \

# @markdown ---

# @markdown Decide if you want to create a video
do_create_video = True # @param {type: "boolean"}
if do_create_video:
    # @markdown Set the FPS of the output video
    fps = 24.0 # @param {type: "number"}
    fps = float(fps)
    # @markdown Give an output path for the video
    video_path = "/content/interpolated.mp4" # @param {type: "string"}

    frames_pattern = os.path.join(input_dir, 'interpolated_frames', "*")
    frames_to_video(
        frames_pattern,
        video_path,
        fps=fps,
    )
    show_video(video_path)

In [ ]:
# @title Interpolate (n-times) between video frames
%cd /content/frame-interpolation

import os


# @markdown Provide the path to a video file:
video_path = "/content/gdrive/MyDrive/projects/ML4ME/vision/frame_interpolation/film/video.mp4" # @param {type: "string"}
# @markdown Set an output directory for the video frames:
frames_dir = "/content/frames"

!rm -r $frames_dir
video_to_frames(video_path, frames_dir)

# @markdown Specify the number of interpolated frames:
n_times_to_interpolate = 6 # @param {type: "number"}

!python -m eval.interpolator_cli  \
   --pattern $frames_dir \
   --model_path $local_pretrained_models_dir/film_net/Style/saved_model \
   --times_to_interpolate $n_times_to_interpolate \

# @markdown ---

# @markdown Decide if you want to create a video
do_create_video = True # @param {type: "boolean"}
if do_create_video:
    # @markdown Set the FPS of the output video
    fps = 24.0 # @param {type: "number"}
    fps = float(fps)
    # @markdown Give an output path for the video
    video_path = "/content/interpolated.mp4" # @param {type: "string"}

    frames_pattern = os.path.join(frames_dir, 'interpolated_frames', "*")
    frames_to_video(
        frames_pattern,
        video_path,
        fps=fps,
    )
    show_video(video_path)